In [1]:
## Importieren der erforderlichen Bibliotheken
# Pyhton Version 3.11.1
# Selenium Version 4.9.1
# webdriver_manager Version 3.8.6
# pandas Version  2.0.1
# datetime Version 5.1
# sqlalchemy Version 2.0.13
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
# import pandas as pd
import datetime

import tools
import pandas as pd
tabelle_rohdaten = "jobs.rohdaten"
scraper_name = "stepstone"


## Der Scraper
def stepstone_scraper(jobtitel, suchort='Deutschland'):
    '''
    Der eigentliche Scraper für die Seite

    :param jobtitel: Der Titel des Jobs, nach dem gesucht werden soll.
    :param suchort: Der Ort, an dem nach Jobs gesucht werden soll. Standardwert ist 'Deutschland'.
    '''

    tools.schreibe_log_file(scraper_name, f'suche nach {jobtitel}')

    con = tools.connect_db()

    # ChromeOptions erstellen
    chrome_options = webdriver.ChromeOptions()
    #Headless-Modus aktivieren
    #chrome_options.add_argument("--headless")

    # Browserfenster öffnen nach Einstellung in den Optionen
    try:
        driver = webdriver.Chrome(options=chrome_options)
        tools.wartezeit(1, 1)  # Wartezeit das Browser geladen ist
        tools.schreibe_log_file(scraper_name, 'Browser geoeffnet')
    except:
        tools.schreibe_log_file(scraper_name, 'Browser konnte nicht geoeffnet werden')
        tools.schreibe_e_mail(scraper_name,
                              f'Browser konnte nicht geoeffnet werden: -- {datetime.datetime.now().strftime("%d.%m.%Y, %H:%M:%S")} --')

    # Webseite aufrufen
    driver.get("https://www.stepstone.de/")

    # Fenster Maximieren
    driver.maximize_window()  # evtl. im Headless-Modus nicht machbar

    # Cookies akzeptieren
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="ccmgt_explicit_accept"]').click()
    tools.wartezeit(0.5, 1)

    ## Suchfelder finden, anwählen und befüllen
    # Suchfeld Jobbezeichnung -> finden und befüllen
    suchfeld_jobtitel = driver.find_element(By.XPATH, '//*[@id="stepstone-autocomplete-162"]')
    suchfeld_jobtitel.send_keys(jobtitel)
    tools.wartezeit(0.5, 1)

    # Suchfeld Ort -> finden und befüllen
    suchfeld_ort = driver.find_element(By.XPATH, '//*[@id="stepstone-form-element-173-input"]')
    suchfeld_ort.send_keys(suchort)
    tools.wartezeit(0.5, 1)

    # Anfrage mit Enter/Return abschicken
    suchfeld_ort.send_keys(Keys.RETURN)
    tools.wartezeit(0.5, 1)

    ## nach Datum sortieren
    # erst Filterfeld dann Datum klicken
    driver.find_element(By.CLASS_NAME, 'res-1vztcyh').click()
    tools.wartezeit(0.5, 1)
    driver.find_element(By.XPATH, '//*[@id="date"]/span/span[2]').click()
    time.sleep(3)

    # Leere Liste erstellen für die Links
    link_liste_scraped = []

    ## Namen für verschiedene XPATH, CLASS, CSS, etc.
    # name für den Container der alle Stellenanzeigen enthält
    class_name_anzeige_link = 'res-kyg8or'

    # name für den Container für alle URLs der einzelnen Stellenanzeigen
    # da es meherer Namen gibt, mit Schleife herausfinde welche es gibt
    for class_name in ['res-3yv1ty', 'res-2cltag']:
        if driver.find_elements(By.CLASS_NAME, class_name) != []:
            class_name_url_link = class_name

    ## Aus den Stellenanzeigen die Links zu den Stellenanzeigen finden und in eine Liste packen
    # alle Stellenanzeigen auf der seite finden
    anzeigen = driver.find_elements(By.CLASS_NAME, class_name_anzeige_link)

    # in jeder Stellenazige die URL heraussuchen
    for anzeige in anzeigen:
        try:
            for link in anzeige.find_elements(By.CLASS_NAME, class_name_url_link):
                link_liste_scraped.append(link.get_attribute('href'))
        except:
            tools.schreibe_log_file(scraper_name, f'Link in Anzeige auf erster Seite nicht gefunden')
            continue

    # weitere Seiten aufrufen
    for i in range(9):

        # Link zur nächsten seite finden und aufrufen
        try:
            driver.get(driver.find_elements(By.CLASS_NAME, 'res-1w7ajks')[1].get_attribute('href'))
        except:
            tools.schreibe_log_file(scraper_name, 'Class-Name nicht gefunden für nächsten seite der Stellenanzeigen')
            pass
        #Warten auf den Aufbau der Seite
        time.sleep(2)

        # alle Stellenanzeigen auf der seite finden
        anzeigen = driver.find_elements(By.CLASS_NAME, class_name_anzeige_link)

        # in jeder Stellenazige die URL heraussuchen
        for anzeige in anzeigen:
            for link in anzeige.find_elements(By.CLASS_NAME, class_name_url_link):
                link_liste_scraped.append(link.get_attribute('href'))

    # überprüfen ob link_liste_scraped leer ist da evtl. die Class name geändert wurde
    if len(link_liste_scraped) == 0:
        tools.schreibe_log_file(scraper_name, f'Keine Links auf gefunden: Jobtitel => {jobtitel}')
        tools.schreibe_e_mail(scraper_name,
                              f'Keine Links auf Webseite gefunden: Jobtitel => {jobtitel}\n\n Hinweis: Classenname überprüfen für alle Stellenanzeigen!')
        return
    else:
        pass

    # Duplikate entfernen
    link_liste_scraped_ohne_duplikate = list(set(link_liste_scraped))
    tools.schreibe_log_file(scraper_name,
                            f'Es wurden {len(link_liste_scraped) - len(link_liste_scraped_ohne_duplikate)} Duplikate in der "Link Liste" entfernt')
    tools.schreibe_log_file(scraper_name, "link liste erstellt")

    # Lese die bestehenden URLs aus der Datenbank
    existing_urls = pd.read_sql_query(f"SELECT url FROM {tabelle_rohdaten}", con)
    tools.schreibe_log_file(scraper_name, f'{len(existing_urls)} in der datenbank bereits vorhanden')

    # Filtere den DataFrame, um nur neue URLs zu behalten
    link_liste = [url for url in link_liste_scraped_ohne_duplikate if url not in existing_urls['url'].values]

    # Listen für für den DataFrame
    seiten_inhalt_html_liste = []
    seiten_inhalt_liste = []
    URL_Liste = []
    Datum_Liste = []

    ## Scrapen
    try:
        tools.schreibe_log_file(scraper_name, 'Abfrage begonnen')
        for link in link_liste:

            # Aufrufen der Seite
            try:
                driver.get(link)
                tools.wartezeit(0.5, 2)
            except:
                tools.schreibe_log_file(scraper_name, f'Fehler beim aufrufen der URL: {link}')

            # Prüfen ob ein bestimmter Text auf der Seite steht, der auf die nicht mehr existierende Anzeige hindeutet
            try:
                stellenanzeige_pruefen = driver.find_element(By.CLASS_NAME, 'listing-content-provider-1qtvd67').text
            except:
                stellenanzeige_pruefen = ""

            # Prüfen ob Stellenanzeige existiert oder nicht
            if stellenanzeige_pruefen != 'Diese Stellenanzeige ist nicht mehr verfügbar.':

                ## Grunddaten scrapen
                seiten_inhalt_html_liste.append(
                    driver.find_element(By.CLASS_NAME, 'reb-main').get_attribute('innerHTML'))
                seiten_inhalt_liste.append(driver.find_element(By.CLASS_NAME, 'reb-main').text)

                # URL der Stellenanzeige
                URL_Liste.append(link)

                # Zeitstempel
                Datum_Liste.append(datetime.datetime.now())

                tools.schreibe_log_file(scraper_name, f'Daten wurden gezogen: {link}')
            else:
                tools.schreibe_log_file(scraper_name, f'Stellenanzeige nicht verfügbar:  {link}')
    except:
        tools.schreibe_log_file(scraper_name, "Abfrage abgebrochen !!!")
        tools.schreibe_e_mail(scraper_name, f"Die Abfrage wurde abgebrochen. -- {datetime.datetime.now()} --")
    tools.schreibe_log_file(scraper_name, 'Abfrage beendet')

    # Schließen des Browsers
    driver.quit()
    tools.schreibe_log_file(scraper_name, 'Browser geschlossen')
    time.sleep(3)

    # Prüfen ob die Listen gleich lang sind
    if len(set(map(len, [seiten_inhalt_html_liste,
                         seiten_inhalt_liste,
                         URL_Liste,
                         Datum_Liste]))) > 1:
        tools.schreibe_log_file(scraper_name, 'Listen sind verschieden lang')
        tools.schreibe_e_mail(scraper_name, f'Listen sind nicht gleich lang  -- {datetime.datetime.now()} --')
        return
    else:
        # DataFrame erstellen
        df = pd.DataFrame({"seite": "stepstone",
                           "seiten_inhalt_html": seiten_inhalt_html_liste,
                           "seiten_inhalt": seiten_inhalt_liste,
                           "url": URL_Liste,
                           "datum": Datum_Liste,
                           "storno": False})

    ## Daten in die Datenbank einfügen
    # Schreibe den bereinigten DataFrame in die Datenbank
    try:
        df.to_sql(name=tabelle_rohdaten.split('.')[1],
                  schema=tabelle_rohdaten.split('.')[0],
                  con=con, if_exists='append', index=False, )
        tools.schreibe_log_file(scraper_name, f'Es wurden {len(df)} Daten von Stepstone hinzugefügt')
    except:
        tools.schreibe_log_file(scraper_name, 'Datenbank konnte nicht gefüllt werden => Dataframe fehlt')
        tools.schreibe_e_mail(scraper_name,
                              f'Datenbank konnte nicht gefüllt werden => Dataframe fehlt  -- {datetime.datetime.now()} --')

    # Verbindung zur SQL-Datenbank schließen
    con.dispose()


print("fertig")

fertig


In [4]:
stepstone_scraper("Data Analyst")

gaierror: [Errno 10109] getaddrinfo failed

In [10]:
# Ich arbeite mit zwei Computern, und leider hat die Datei für die Suchbegriffe (Jobtitel) jeweils ein anderen Speicherort
try:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                 sheet_name='Stepstone')
except:
    suchbegriffe = pd.read_excel(r"C:\Users\Admin\DreamJobs\Benötigte Dateien\Jobtitel.xlsx",
                                 sheet_name='Stepstone')
display(suchbegriffe)
print(f'{len(suchbegriffe)} Jobtitel gefunden')
for i, jobtitel in enumerate(suchbegriffe.Jobtitel):
    print(f'{i}: Suche nach "{jobtitel}", Start: {datetime.datetime.now().strftime("%H:%M:%S")} Uhr')
    scraper(jobtitel)
print("")
print("fertig")

,Jobtitel
0,Data Analyst
1,Junior Data Analyst
2,Data Scientist & Machine Learning Engineer
3,Data Expert
4,Data Warehouse Developer
5,(Junior) Data Analyst
6,Data Scientist
7,Customer Data Analyst / Engineer
8,Big Data Engineer
9,Data Consultant


13 Jobtitel gefunden
0: Suche nach "Data Analyst", Start: 09:18:29 Uhr


NameError: name 'scraper' is not defined